In [1]:
import re
import shutil
from github3 import login
from git import Repo
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
%cd ~/Documents/projects/nerve/python/
from nerve.core.model import Nerve
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.utils import execute_subprocess

/Users/alex/Documents/projects/nerve/python


In [2]:
config_path = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
nerve = Nerve(config_path)
name = 'nerve-test-repo'
proj = os.path.join(nerve['project-root'], name)

In [4]:
# nerve.delete(name, path=proj, local=True)
nerve.create(name)
nerve.clone(name)

KeyError: 'uri'